# Section 2 - SymPy Expressions and Solvers

Objectives:
- Quick refresher on SymPy basics
- Equation and Function data types
- Differential equation solvers

Start by running the import statement below (this must be done every time you start up Jupyter Notebook or Google Colab).

In [2]:
import sympy as sp

## 2.1 SymPy Refresher

SymPy (symbolic Python) is a package containing the **symbol** and **symbolic expression** data types, as well as many functions used to manipulate them.

In SymPy, your starting point is (nearly) always to create a symbolic variable:

In [3]:
x = sp.symbols('x')
y, z = sp.symbols('y z')                 # For multiple variables. Notice the space inside the string instead of a comma

iamasymbol = sp.symbols('iamasymbol')    # You can use a whole word- it just isn't going to look good.

theta = sp.symbols('theta')
theta                                    # With SymPy's LaTeX printing mode, the actual Greek letters get displayed!

theta

Often, you'll want to restrict the domain for a variable- *by default, symbols are assumed to be complex variables*, so any time you use a solver, it will consider complex solutions unless you tell it otherwise. Here are some ways you can do so:

In [4]:
x = sp.symbols('x', real = True) # x in R
x = sp.symbols('x', positive = True) # x in R+
x = sp.symbols('x', integer = True) # x in Z
x = sp.symbols('x', integer = True, positive = True) # x in N

To see all assumptions about a symbolic variable, print the **assumptions0** attribute of the variable:

In [5]:
print(x.assumptions0)

{'integer': True, 'extended_real': True, 'noninteger': False, 'algebraic': True, 'finite': True, 'rational': True, 'complex': True, 'transcendental': False, 'irrational': False, 'real': True, 'infinite': False, 'hermitian': True, 'commutative': True, 'imaginary': False, 'positive': True, 'zero': False, 'extended_nonpositive': False, 'negative': False, 'extended_nonnegative': True, 'nonzero': True, 'extended_negative': False, 'extended_positive': True, 'nonnegative': True, 'extended_nonzero': True, 'nonpositive': False}


Using symbols, you can define a **symbolic expression**. Basic arithmetic for symbols is the same as for numbers, just tweaked behind the scenes to handle these new data types:

In [6]:
f = 3 * x * y + x**y - x % y + x // y
f

3*x*y + x**y - Mod(x, y) + floor(x/y)

**Remember, the Math and NumPy package versions of exp, log, sin, etc. will NOT work with symbolic variables and expressions!**

Use the SymPy versions instead: **sp.exp(x), sp.log(x,base), sp.sin(x), sp.Abs(x), sp.sqrt(x)**, etc.

For a much more comprehensive list, see https://docs.sympy.org/latest/modules/functions/elementary.html

Moreover, SymPy comes with several constants built-in:
- **sp.pi**
- **sp.E** (yes, capitalized)
- **sp.I** (also capitalized)

## 2.2 The Function and Equality (Equation) Data Types

### Functions

Functions (with a capital "F") in SymPy are symbolic representations of a function $f(x)$. They are defined similarly to symbols:

In [ ]:
x, y = sp.symbols('x y')
f = sp.Function('f')(x) # f is a function of x
g = sp.Function('g')(x,y) # g is a function of x and y
g

Note: At some point, you were able to define a Function using sp.symbols with the assumption "cls=Function", but they seem to have removed that method.

For differential equations, you also want representations of the derivative(s) of an unknown $f$:

In [ ]:
fp = sp.Derivative(f,x)
fpp = sp.Derivative(f,x,2) # Second derivative. Replace 2 with n to get nth derivative
dgdx = sp.Derivative(g,x)
d2gdxdy = sp.Derivative(g,x,y) # Second partial derivative w.r.t. x and y
d2gdxdy

For integral equations, you might want this:

In [ ]:
F = sp.Integral(f,x)
F

Note: Function, Derivative, and Integral are capitalized. This is a way to remember these create and manipulate symbolic stand-ins for functions, and are *not* operations to be applied to expressions.

### Equalities

Next, we have the general data type of **relations**. Of primary interest to us today is **Equalities** (what SymPy calls their equation data type). Use **sp.Eq(LHS,RHS)** to create one.

In [ ]:
LHS =  3*x**2 - 12*x
RHS = -9
simpleEq = sp.Eq(LHS,RHS)
simpleEq

In [ ]:
LHS = fp
RHS = f
simpleODE = sp.Eq(LHS,RHS)
simpleODE

You can't do arithmetical operations to both sides like you could by hand, but **simplify** and other algebraic commands will work on Equalities:

In [ ]:
simpleEq.simplify() # or you could put sp.simplify(simpleEq)

**Your turn**: Write code that assembles the ODE $2xf''(x) = f(x)$ as a SymPy Equality.

In [ ]:
x = sp.symbols('x', real = True)



## 2.3 (Ordinary) Differential Equations Solver

SymPy has many solvers, depending on the type of equation.

The syntax for *most* solvers is **sp.solvername(expression, whattosolvefor)**, where the given expression is set equal to 0.

You can also input an Equality: **sp.solvername(equality, whattosolvefor)**.

**Important:** If your variables are not meant to be complex, you should include those assumptions when defining your variables. This gives the solvers more information to help reach the solution(s) faster.

For more information, see https://docs.sympy.org/latest/modules/solvers/solvers.html

For other types of solvers, see https://docs.sympy.org/latest/modules/solvers/index.html

### sp.dsolve()

SymPy's ODE solver. As usual, the input can be an expression to be set equal to 0 or an Equality.

In [14]:
x, y = sp.symbols('x y')
f = sp.Function('f')(x) # f is a function of x
fp = sp.Derivative(f,x)
LHS = fp
RHS = f
simpleODE = sp.Eq(LHS,RHS)
simpleODE

Eq(Derivative(f(x), x), f(x))

In [15]:
sp.dsolve(simpleODE,f)

Eq(f(x), C1*exp(x))

Note that it gave us the most general form, with arbitrary constant $C_1$. Suppose we have initial or boundary conditions for our DE. For example, in the above, let's say $f(1) = e^2$. Then we would set the **ics** variable inside the dsolve command to hold our conditions like so:

In [18]:
sp.dsolve(simpleODE,f, ics = {f.subs(x,1):sp.E**2})

Eq(f(x), E*exp(x))

For more than one condition, separate them by commas inside the {}.

It can also solve a system of ODEs. For more information, see https://docs.sympy.org/latest/modules/solvers/ode.html

There is also **sp.pdsolve()** for PDEs:

In [26]:
x, t = sp.symbols('x t', real = True)
k = sp.symbols('k', positive = True)
f = sp.Function('f') # This line and the next demonstrate another means of defining a Function and its input variables.
u = f(x, t)
ux = u.diff(x)
ut = u.diff(t)
eq = sp.Eq(ut, ux)
eq

Eq(Derivative(f(x, t), t), Derivative(f(x, t), x))

In [28]:
sp.pdsolve(eq)

Eq(f(x, t), F(t + x))

To solve more complicated PDEs often requires more specialized tools than this. See https://docs.sympy.org/latest/modules/solvers/pde.html for a more detailed list of all available tools for classifying and solving PDEs using SymPy.

### Solving a system of ODEs

You still use sp.dsolve, but the equations (or expressions to set equal to 0) given need to be in a list, that is, arranged in the form **[Eq1, Eq2, ... EqN]**

In [15]:
x = sp.symbols('x')
f = sp.Function('f')(x)
g = sp.Function('g')(x)

eqs = [sp.Eq(sp.Derivative(f,x), g), sp.Eq(sp.Derivative(g,x), f)]

solns = sp.dsolve(eqs)
solns

[Eq(f(x), -C1*exp(-x) + C2*exp(x)), Eq(g(x), C1*exp(-x) + C2*exp(x))]

To retrieve an individual solution, use Python indexing (starting with 0).

In [16]:
solns[0]

Eq(f(x), -C1*exp(-x) + C2*exp(x))

In [17]:
solns[1]

Eq(g(x), C1*exp(-x) + C2*exp(x))

## 2.4 (Optional) Laplace Transforms

SymPy also has both the Laplace transform and the Dirac Delta ($\delta_{k=0}$) built in, in case you want to use SymPy to demonstrate these concepts (or give them an assignment on them using Python).

The syntax for the Dirac Delta is **sp.DiracDelta(x)**. Plugging in 0 will just give the symbolic expression $\delta(0)$ and not return infinity, though plugging in anything nonzero will return zero. The documentation gives a more thorough description of how it works: https://docs.sympy.org/latest/modules/functions/special.html#sympy.functions.special.delta_functions.DiracDelta

For Laplace transforms, the syntax is

**sp.laplace_transform(function, t, s)**

where $t$ is the independent variable of the function and $s$ is the desired independent variable of the transform.

In [30]:
t, s = sp.symbols('t s')

sp.laplace_transform(t**4, t, s)

(24/s**5, 0, True)

The output has 3 parts:

- First, the transform expression.
- The Laplace transform converges absolutely on a half-plane $a < re(s)$ for some $a$. The second part of the output is that $a$.
- The third part is some additional conditions on the validity of the transform, as seen below. For transforms with no other conditions besides the half-plane restriction, the third part is just "True".

In [33]:
a = sp.symbols('a')
sp.laplace_transform(t**a, t, s)

(gamma(a + 1)/(s*s**a), 0, re(a) > -1)

Since the output is a tuple (like a list, but with () instead of []), we can use Python indexing to refer to a single part:

In [34]:
sp.laplace_transform(t**a, t, s)[0]

gamma(a + 1)/(s*s**a)

Here's an example involving the Dirac Delta:

In [35]:
sp.laplace_transform(sp.DiracDelta(t)-a*sp.exp(-a*t), t, s)

(s/(a + s), Max(0, -a), True)

## 2.5 Troubleshooting

Remember: sometimes the simplest explanation for an issue is the best. Look for typos (especially misplaced parenthesis or brackets), commands not being called properly, etc.

1) The solver never finishes, or it gives a NotImplementedError, or the error message says it cannot solve the equation.

It's sad but true- SymPy can't solve every equation, but the same can be said about Mathematica, the MATLAB symbolic toolbox, or the TI-Nspire. You can try these things to see if you can coax a solution:
- Give the solver as much information as you can- include all relevant assumptions in your symbol definitions.
- Play with the simplify/expand commands.

2) ValueError

This error usually occurs when the parameters given to a solver don't match the required form. Double-check your syntax. Remember that the initial condition parameter **ics** expects a dictionary, which is wrapped in {}.

Finally, a reminder of a couple common issues with SymPy in general that were discussed in the introductory workshop:

- Symbolic expressions do NOT work with the Math or NumPy packages.

- SymPy sometimes gets shaken by decimal powers.

The second one is best explained through example:

In [ ]:
x = sp.symbols('x', real = True)

f = (x**2 - 1)**(3/2)
sp.pprint(f)
sp.integrate(f,x)

Notice that the 3/2 was turned into 1.5 upon storing the expression to $f$. Python follows order of operations, whether we like it or not. 3/2 evaluates to a float, and SymPy isn't the greatest at converting a float to a fraction, but you can use **sp.Rational(num,denom)** to turn 3/2 into a symbolic expression to keep it from becoming a float.

In [ ]:
f = (x**2 - 1)**sp.Rational(3/2)
sp.integrate(f,x)

Another approach is to turn the 3 into a symbolic expression using **sp.S()**. Dividing a symbolic expression by an integer will give another symbolic expression.

In [ ]:
sp.S(3) / 2

To see the difference, check the data types:

In [ ]:
print(type(3))
print(type(sp.S(3)))